In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, balanced_accuracy_score
from dgl.dataloading import GraphDataLoader

from utils import GraphImageData
from train import cRT_training

/tmp/ipykernel_158791/707430630.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# load dataset
graph_dir = '../merged_data_csv1'
img_dir = '../PWI_tumour_cropped'
clinic_dir = './IDH_mask_avail_clinic.csv'
img_size = (54, 54, 54)
time_len = 45
dataset = GraphImageData(graph_dir=graph_dir, imgs_dir=img_dir, clinic_dir=clinic_dir, img_size=img_size, time_len=time_len)
print(f"\tLoaded Dataset from: graphs '{graph_dir}' & images '{img_dir}' of size: {len(dataset)} ...\n")

Done loading data from cached files.
	Loaded Dataset from: graphs '../merged_data_csv1' & images '../PWI_tumour_cropped' of size: 444 ...



In [3]:
g0, img0, label = dataset[0]
(g0, img0.shape, label)

(Graph(num_nodes={'roi': 424},
       num_edges={('roi', 'spatial', 'roi'): 4206, ('roi', 'temporal', 'roi'): 36115},
       metagraph=[('roi', 'roi', 'spatial'), ('roi', 'roi', 'temporal')]),
 torch.Size([45, 1, 54, 54, 54]),
 tensor(0.))

In [62]:
def recombine_dataset(dataset):
    
    graphs = []
    imgs = []
    labels = []
    original_dataset = []
    
    for g, img, label in dataset:
        graphs.append(g)
        imgs.append(img)
        labels.append(label)
        original_dataset.append((g, img, label))
    
    minority_index = list(torch.where(torch.tensor(labels)==1)[0])
    
    recombine_sample = []

    for i in range(len(minority_index)):
        for j in range(len(minority_index)):
            if i == j:
                continue
            g_index = minority_index[i]
            img_index = minority_index[j]
            recombine_sample.append((graphs[g_index], imgs[img_index], labels[g_index]))
    
    import random
    num_to_select = len(labels) - len(minority_index) - len(minority_index)
    recombine_sample = random.sample(recombine_sample, num_to_select)
    
    recombine_sample = original_dataset + recombine_sample
    
    return recombine_sample

In [63]:
recombine_sample = recombine_dataset(dataset)
len(recombine_sample)

828

In [60]:
[1, 2 ,3] + [4, 5, 6]

[1, 2, 3, 4, 5, 6]

In [47]:
graphs = []
imgs = []
labels = []

for g, img, label in dataset:
    graphs.append(g)
    imgs.append(img)
    labels.append(label)

In [52]:
import torch

minority_index = torch.where(torch.tensor(labels)==1)[0]
minority_index = list(minority_index)

In [57]:
len(labels) - len(minority_index)

414

In [54]:
recombine_sample = []

for i in range(len(minority_index)):
    for j in range(len(minority_index)):
        if i == j:
            continue
        g_index = minority_index[i]
        img_index = minority_index[j]
        recombine_sample.append((graphs[g_index], imgs[img_index], labels[g_index]))
len(recombine_sample)

870

In [56]:
print([label for _, _, label in recombine_sample])

[tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), ten

In [12]:
from dataloaders import graph_img_dataloader

batch_size = 2
seed = 42

train_dataloader, val_dataloader, test_dataloader = graph_img_dataloader(dataset=dataset,
                                                                        retrain=False,
                                                                        train_batch_size = batch_size,
                                                                        val_batch_size=batch_size,
                                                                        test_batch_size=batch_size,
                                                                        seed=seed)

In [5]:
from Perf_GNN import PerfGAT

input_dim = 45
node_hidden_dim = 16
edge_hidden_dim = 8
num_layer = 2
dropout_prob = 0.15
edge_feat = True

model = PerfGAT(input_dim, 1, node_hidden_dim, edge_hidden_dim, num_layer=num_layer, dropout_prob=dropout_prob,
                                edge_feat=edge_feat)

In [13]:
import torch

for graphs, imgs, labels in train_dataloader:
    print(graphs)
    print(imgs.shape)
    print(labels.shape)
    node_feat = graphs.ndata['feat']
    edge_feat = torch.unsqueeze(graphs.edges['temporal'].data['feat'], 1)
    outputs = model(graphs, node_feat, edge_feat, imgs, keep_node=True,
                                            rm_perc=0.4, add_perc=0.01)
    for out in outputs:
        print(out.shape)
    break

    

Graph(num_nodes={'roi': 848},
      num_edges={('roi', 'spatial', 'roi'): 8412, ('roi', 'temporal', 'roi'): 71970},
      metagraph=[('roi', 'roi', 'spatial'), ('roi', 'roi', 'temporal')])
torch.Size([2, 45, 1, 54, 54, 54])
torch.Size([2])
torch.Size([2, 48])
torch.Size([2, 424, 48])
torch.Size([2, 424, 48])
torch.Size([2, 48])


In [7]:
import os.path

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ["DGLBACKEND"] = "pytorch"

import dgl

graphs_ubatch = dgl.unbatch(graphs)
graphs_ubatch

[Graph(num_nodes={'roi': 424},
       num_edges={('roi', 'spatial', 'roi'): 4206, ('roi', 'temporal', 'roi'): 36028},
       metagraph=[('roi', 'roi', 'spatial'), ('roi', 'roi', 'temporal')]),
 Graph(num_nodes={'roi': 424},
       num_edges={('roi', 'spatial', 'roi'): 4206, ('roi', 'temporal', 'roi'): 35987},
       metagraph=[('roi', 'roi', 'spatial'), ('roi', 'roi', 'temporal')])]

In [14]:
img_feats, temporal_feats, spatial_feats, _ = outputs

In [19]:
spatial_feats.shape

torch.Size([2, 424, 48])

In [17]:
import torch.nn as nn

cosinesim = nn.CosineSimilarity(dim=1)
similarity = []
for img_feat, spatial_feat in zip(img_feats, spatial_feats):
    sim = cosinesim(spatial_feat, img_feat)
    similarity.append(sim)
similarity = torch.stack(similarity)
similarity.shape

torch.Size([2, 424])

In [22]:
torch.stack([similarity]*spatial_feats.shape[-1], dim=2).shape

torch.Size([2, 424, 48])

In [23]:
(torch.stack([similarity]*spatial_feats.shape[-1], dim=2) * spatial_feats).shape

torch.Size([2, 424, 48])

In [35]:
similarity.shape

torch.Size([2, 424])

In [24]:
weighted_sfeats = torch.stack([similarity]*spatial_feats.shape[-1], dim=2) * spatial_feats
z = torch.stack([weighted_sfeats, temporal_feats], dim=1)
z.shape

torch.Size([2, 2, 424, 48])

In [26]:
project = nn.Sequential(
            nn.Linear(48, 128),
            nn.Tanh(),
            nn.Linear(128, 1, bias=False),
        )
w_d = project(z).mean(2)
w_d.shape

torch.Size([2, 2, 1])

In [39]:
beta = torch.softmax(w_d, dim=1)
beta.shape

torch.Size([2, 2, 1])

In [37]:
beta = beta.expand((z.shape[0],) + beta.shape)
beta.shape

torch.Size([2, 2, 2, 1])

In [41]:
beta = beta.unsqueeze(2)
beta.shape

torch.Size([2, 2, 1, 1])

In [43]:
(z * beta).shape

torch.Size([2, 2, 424, 48])

In [44]:
out = torch.sum((z*beta), dim=1)
out.shape

torch.Size([2, 424, 48])

In [45]:

import torch.nn as nn

class dual_attn_fusion(nn.Module):
    def __init__(self, in_size=48, hidden_size=128):
        super(dual_attn_fusion, self).__init__()
        
        self.cosinesim = nn.CosineSimilarity(dim=1)
        
        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False),
        )
        
    def forward(self, img_feats, spatial_feats, temporal_feats):
        # img_feats: [batch, 48], graph_feats: [batch, nodes, 48]
        
        # Node Attention
        similarity = []
        for img_feat, spatial_feat in zip(img_feats, spatial_feats):
            
            sim = self.cosinesim(spatial_feat, img_feat)
            similarity.append(sim)
        
        similarity = torch.stack(similarity)
        
        weighted_sfeats = torch.stack([similarity]*spatial_feats.shape[-1], dim=2) * spatial_feats
        
        z = torch.stack([weighted_sfeats, temporal_feats], dim=1)
        
        # Semantic Attention
        w_d = self.project(z).mean(2)
        beta = torch.softmax(w_d, dim=1)
        beta = beta.unsqueeze(2)
        out = torch.mean(torch.sum((z*beta), dim=1), dim=1)
        
        return out
        
        

In [46]:
fusion = dual_attn_fusion()

out = fusion(img_feats, spatial_feats, temporal_feats)
out.shape

torch.Size([2, 48])